In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/home/shtoshni/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [5]:
data = dataset['validation']

In [49]:
dataset['validation'][:3]

{'article': ['(CNN)Singer-songwriter David Crosby hit a jogger with his car Sunday evening, a spokesman said. The accident happened in Santa Ynez, California, near where Crosby lives. Crosby was driving at approximately 50 mph when he struck the jogger, according to California Highway Patrol Spokesman Don Clotworthy. The posted speed limit was 55. The jogger suffered multiple fractures, and was airlifted to a hospital in Santa Barbara, Clotworthy said. His injuries are not believed to be life threatening. "Mr. Crosby was cooperative with authorities and he was not impaired or intoxicated in any way. Mr. Crosby did not see the jogger because of the sun," said Clotworthy. According to the spokesman, the jogger and Crosby were on the same side of the road. Pedestrians are supposed to be on the left side of the road walking toward traffic, Clotworthy said. Joggers are considered pedestrians. Crosby is known for weaving multilayered harmonies over sweet melodies. He belongs to the celebrate

In [6]:
idx = 4

print(data[idx]['article'])
print()
print(data[idx]['highlights'])

Coral Gables, Florida (CNN)Former President Bill Clinton on Saturday defended his family foundation's practice of taking money from foreign countries, arguing that while he doesn't agree with all of the policies of countries that contributed, he feels the foundation has "done a lot more good than harm." The Clinton Foundation admitted last month that a 2010 donation from the Algerian government was not properly approved under the guidelines the Obama administration put in place with the foundation when Hillary Clinton became secretary of state in 2009. Bill Clinton defended the donations as something that went to worthwhile projects. "The UAE [United Arab Emirates] gave us money. Do we agree with everything they do? No. But they are helping us fight ISIS and they built a great university with NYU open to people around the world," Clinton said at a foundation event in Florida. "Do I agree with all the foreign policy of Saudi Arabia? No." Saudi Arabia, the United Arab Emirates and Oman a

In [7]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [10]:
import sys
sys.path.append('/home/shtoshni/Research/long-doc-coref/src')

from inference.inference import Inference
model_loc = "/home/shtoshni/Research/long-doc-coref/models/umem_singleton_90K/model.pth"
model = Inference(model_loc)



In [11]:
model.model.max_span_width = 10
model.model.memory_net.new_ent_score = 0

In [37]:
dir(nlp(instance['highlights']).ents[0])

['_',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_fix_dep_copy',
 '_vector',
 '_vector_norm',
 'as_doc',
 'char_span',
 'conjuncts',
 'doc',
 'end',
 'end_char',
 'ent_id',
 'ent_id_',
 'ents',
 'get_extension',
 'get_lca_matrix',
 'has_extension',
 'has_vector',
 'kb_id',
 'kb_id_',
 'label',
 'label_',
 'lefts',
 'lemma_',
 'n_lefts',
 'n_rights',
 'noun_chunks',
 'orth_',
 'remove_extension',
 'rights',
 'root',
 'sent',
 'sentiment',
 'set_extension',
 'similarity',
 'start',
 'start_char',
 'subtree',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'vector',
 'vector_norm',
 'vocab']

In [48]:
nlp("October's").doc[1]

's

In [36]:
[str(word) for word in nlp(instance['highlights']).doc]

['Spokesperson',
 ':',
 'Experts',
 'are',
 'investigating',
 'how',
 'the',
 'UK',
 'military',
 'health',
 'care',
 'worker',
 'got',
 'Ebola',
 '.',
 '\n',
 'It',
 'is',
 'being',
 'decided',
 'if',
 'the',
 'military',
 'worker',
 'infected',
 'in',
 'Sierra',
 'Leone',
 'will',
 'return',
 'to',
 'England',
 '.',
 '\n',
 'There',
 'have',
 'been',
 'some',
 '24,000',
 'reported',
 'cases',
 'and',
 '10,000',
 'deaths',
 'in',
 'the',
 'latest',
 'Ebola',
 'outbreak',
 '.']

In [12]:
import spacy 
nlp = spacy.load('en_core_web_trf')

In [24]:
for idx, instance in enumerate(dataset['validation']):
    summary = instance['highlights']
    document = instance['article']
    output_dict = model.perform_coreference([instance['highlights'], instance['article']])
    
    summary_len = output_dict['tokenized_doc']['part_lens'][0]
    
    summary_ents = nlp(instance['highlights']).ents
    summary_ents = set([str(ent) for ent in summary_ents])
    
    
    doc_ents = nlp(instance['article']).ents
    doc_ents = set([str(ent) for ent in doc_ents])
    
    ent_diff = summary_ents.difference(doc_ents)
    
    if len(ent_diff):
        print(f"{idx} NER output:", ent_diff)
#         print(summary_ents)
        print()
    
   # Only keep capitalized singletons
    found_ner_singleton = False
    
    
    singletons = [cluster[0] for cluster in output_dict['clusters'] 
                  if len(cluster) == 1 and cluster[0][0][0] < summary_len ]
    
    ner_singletons = []
    for singleton in singletons:        
        if singleton[1] == 'It':
            continue
#         ner_singletons.append(singleton)  
        singleton_added = False
        words = singleton[1].split()
        for word in words:
            if word[0].isupper():
                ner_singletons.append(singleton)
                singleton_added = True
                break
                
        if (singleton[1] in summary_ents) and (not singleton_added):
            ner_singletons.append(singleton)
    
    # Also check that the mention is not a super mention of a smaller mention
    mentions = output_dict['mentions']
    filtered_singletons = []
    for singleton in singletons:
        singleton_start, singleton_end = singleton[0]
        
        super_string = False
        for ment_start, ment_end in mentions:
            if singleton_start <= ment_start and singleton_end >= ment_end:
                # Check it's not the same mention
                if singleton_start == ment_start and singleton_end == ment_end:
                    continue
                else:
                    super_string = True
                    break
                    
        if not super_string:
            filtered_singletons.append(singleton[1])        
    
                
    if len(filtered_singletons):
        print(f"{idx} Coref output:", filtered_singletons)
        print(ner_singletons)
#         print(output_dict['clusters'] )
        print()
        
    
    if idx >= 8:
        break
        
    if idx >= 6 or idx == 2:
        print(instance['article'])
        print()
        print(instance['highlights'])
        print()
        
    

1 NER output: {'Sigma Alpha Epsilon', 'Yale', 'the University of Oklahoma', 'Stanford', 'Johns Hopkins'}

1 Coref output: ['Oklahoma', 'Yale', 'recent months']
[((13, 13), 'Oklahoma'), ((24, 31), 'officials at Yale, Stanford and Johns Hopkins'), ((26, 26), 'Yale'), ((26, 31), 'Yale, Stanford and Johns Hopkins'), ((33, 34), 'recent months')]

2 NER output: {'Bulgaria'}

2 Coref output: ['Bulgaria']
[((24, 27), 'relics found in Bulgaria'), ((27, 27), 'Bulgaria'), ((30, 35), 'the bones of John the Baptist')]

(CNN)I'm Candida Moss and I am professor of New Testament and Early Christianity at the University of Notre Dame. I was an adviser on the "True Cross" episode and served as one of the many on-camera experts in CNN's "Finding Jesus" series, which currently airs on Sundays. Viewers were invited to tweet and post their questions on the "Finding Jesus" Facebook page during the show. Below are some of the more interesting questions and my answers to them. My apologies to everyone I didn't

7 Coref output: ['the death penalty', 'said']
[]

(CNN)Debra Milke spent 22 years on death row, convicted of conspiring with two other men to kill her son allegedly for an insurance payout. On Monday, a judge ruled that the Arizona woman is innocent and dismissed all charges against her. This makes Milke only the second woman exonerated from death row in the United States. More importantly, the judge's decision finally clears Milke after years of legal back-and-forth in a case where she steadfastly maintained her innocence. Key to the case's dismissal was prosecutorial misconduct, mainly that of a detective, Armando Saldate, who said Milke confessed to the crime to him -- even though there was no witness or recording. Prosecutors withheld from the jury Saldate's personnel record which showed instances of misconduct in other cases, including lying under oath. The two men with whom Milke was accused of conspiring were tried separately and are still on death row. A day after seeing Santa 

In [23]:
summary_ents = nlp(instance['highlights']).ents
summary_ents[0].doc

Spokesperson: Experts are investigating how the UK military health care worker got Ebola .
It is being decided if the military worker infected in Sierra Leone will return to England .
There have been some 24,000 reported cases and 10,000 deaths in the latest Ebola outbreak .

In [ ]:
# for idx, instance in enumerate(dataset['validation']):
#     output_dict = model.perform_coreference([instance['article'], instance['highlights']])
#     article_len = output_dict['tokenized_doc']['part_lens'][0]
    
#     summary_ents = nlp(instance['highlights']).ents
#     summary_ents = set([str(ent) for ent in summary_ents])
    
#     doc_ents = nlp(instance['article']).ents
#     doc_ents = set([str(ent) for ent in doc_ents])
    
#     ent_diff = summary_ents.difference(doc_ents)
#     if len(ent_diff):
#         print(f"{idx} NER output:", ent_diff)
# #         print(summary_ents)
#         print()
    
# #     print(output_dict['tokenized_doc']['part_lens'])
# #     print(instance['article'])
    
#     singletons = [cluster[0] for cluster in output_dict['clusters'] 
#                   if len(cluster) == 1 and cluster[0][0][0] >= article_len]
    
# #     if len(singletons):
# #         print(singletons)
# #         print()

# #     print(output_dict['clusters'])
# #     print()

#     # Only keep capitalized singletons
#     found_ner_singleton = False
    
#     ner_singletons = []
#     for singleton in singletons:        
#         if singleton[1] == 'It':
#             continue
#         ner_singletons.append(singleton)  
# #         singleton_added = False
# #         words = singleton[1].split()
# #         for word in words:
# #             if word[0].isupper():
# #                 ner_singletons.append(singleton)
# #                 singleton_added = True
# #                 break
                
# #         if (singleton[1] in summary_ents) and (not singleton_added):
# #             ner_singletons.append(singleton)
    
    
#     # Also check that the mention is not a super mention of a smaller mention
#     mentions = output_dict['mentions']
#     filtered_singletons = []
#     for singleton in ner_singletons:
#         singleton_start, singleton_end = singleton[0]
        
#         super_string = False
#         for ment_start, ment_end in mentions:
#             if singleton_start <= ment_start and singleton_end >= ment_end:
#                 # Check it's not the same mention
#                 if singleton_start == ment_start and singleton_end == ment_end:
#                     continue
#                 else:
#                     super_string = True
#                     break
                    
#         if not super_string:
#             filtered_singletons.append(singleton[1])        
    
                
#     if len(filtered_singletons):
#         print(f"{idx} Coref output:", filtered_singletons)
#         print(ner_singletons)
# #         print(output_dict['clusters'] )
#         print()
        
    
#     if idx >= 8:
#         break
        
#     if idx >= 6:
#         print(instance['article'])
#         print()
#         print(instance['highlights'])
#         print()
        
    
    
    